<a href="https://colab.research.google.com/github/Vera-bahval/geoanalytics/blob/main/%D0%9A%D1%83%D1%80%D1%81%D0%BE%D0%B2%D0%B0%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Скачивание датасета

In [ ]:
!pip install datasets transformers

In [ ]:
import pyarrow

In [ ]:
pyarrow.__version__

'14.0.1'

In [ ]:
! pip install pyarrow==14.0.1

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
from datasets import load_dataset
dataset = load_dataset("pasinit/xlwic", "xlwic_fr_fr")

In [ ]:
item = dataset["train"][0]
item

{'id': 'FR_0',
 'context_1': "Comme l'indique le Shuowen Jiezi (dans son commentaire 从戊丁聲), le caractère 成 (chéng) a certainement hérité sa prononciation de 丁 (zhēng).",
 'context_2': 'D’après le dictionnaire étymologique shuowen, cité dans le Dictionnaire Ricci, jie 節 désigne les nœuds du bambou.',
 'target_word': 'ShuoWen',
 'pos': 'N',
 'target_word_location_1': {'char_start': 19, 'char_end': 26},
 'target_word_location_2': {'char_start': 37, 'char_end': 44},
 'language': 'FR',
 'label': 1}

In [ ]:
print(len(item["context_1"].split()))

21


##Модель camembert-base

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.tokenize(item["context_1"])
tokens

['▁Comme',
 '▁l',
 "'",
 'indique',
 '▁le',
 '▁Shu',
 'ow',
 'en',
 '▁J',
 'iez',
 'i',
 '▁(',
 'dans',
 '▁son',
 '▁commentaire',
 '▁',
 '从戊丁聲',
 '),',
 '▁le',
 '▁caractère',
 '▁',
 '成',
 '▁(',
 'ché',
 'ng',
 ')',
 '▁a',
 '▁certainement',
 '▁hérité',
 '▁sa',
 '▁prononciation',
 '▁de',
 '▁',
 '丁',
 '▁(',
 'z',
 'h',
 'ē',
 'ng',
 ').']

In [ ]:
len(tokens)

40

In [ ]:
# def indicate_subtokens(tokenization, char_start, char_end):
#   i = 0
#   offset_mapping = tokenization["offset_mapping"]
#   subtoken_start, subtoken_end = None, None
#   while subtoken_end is None:
#     if subtoken_start is None and char_start in offset_mapping[i]:
#       subtoken_start = i
#     if subtoken_end is None and char_end in offset_mapping[i]:
#       subtoken_end = i+1
#     i += 1
#   return subtoken_start, subtoken_end
def indicate_subtokens(tokenization, char_start, char_end):
  i = 0
  offset_mapping = tokenization["offset_mapping"]
  subtoken_start, subtoken_end = None, None
  while subtoken_end is None:
    if subtoken_start is None and (char_start == offset_mapping[i][0] or char_start == offset_mapping[i][1] or char_start == (offset_mapping[i][0] + offset_mapping[i][1])//2):
      subtoken_start = i
    if subtoken_end is None and (char_end == offset_mapping[i][0] or char_end == offset_mapping[i][1] or char_end == (offset_mapping[i][0] + offset_mapping[i][1])//2):
      subtoken_end = i+1
    i += 1
  return subtoken_start, subtoken_end

In [ ]:
tokenization = tokenizer(item["context_1"], return_offsets_mapping=True)
for key, value in tokenization.items():
    print(key, value)
print(len(tokenization["input_ids"]))
print(len(tokenization["offset_mapping"]))

input_ids [5, 554, 17, 11, 10087, 16, 24714, 3168, 90, 121, 2054, 233, 38, 3367, 58, 2063, 21, 3, 142, 16, 1428, 21, 3, 38, 2264, 4097, 53, 33, 1975, 17467, 77, 29771, 8, 21, 3, 38, 138, 133, 3, 4097, 120, 6]
attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
offset_mapping [(0, 0), (0, 5), (6, 7), (7, 8), (8, 15), (16, 18), (19, 22), (22, 24), (24, 26), (27, 28), (28, 31), (31, 32), (33, 34), (34, 38), (39, 42), (43, 54), (55, 56), (55, 59), (59, 61), (62, 64), (65, 74), (75, 76), (75, 76), (77, 78), (78, 81), (81, 83), (83, 84), (85, 86), (87, 99), (100, 106), (107, 109), (110, 123), (124, 126), (127, 128), (127, 128), (129, 130), (130, 131), (131, 132), (132, 133), (133, 135), (135, 137), (0, 0)]
42
42


In [ ]:
subtoken_start, subtoken_end = indicate_subtokens(tokenization, item['target_word_location_1']['char_start'], item['target_word_location_1']['char_end'])
subtoken_start, subtoken_end

(6, 9)

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("camembert-base").to("cuda")

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
import torch

tensor = torch.LongTensor([tokenization["input_ids"]]).to("cuda")

with torch.no_grad():
    model_output = model(tensor)
model_output.items()

odict_items([('last_hidden_state', tensor([[[ 0.0840,  0.0336, -0.0204,  ..., -0.1185,  0.0306,  0.0075],
         [ 0.0328,  0.1152,  0.3661,  ..., -0.0951,  0.0820,  0.2061],
         [ 0.0274, -0.0907,  0.3735,  ..., -0.1601, -0.1111,  0.0987],
         ...,
         [ 0.0953, -0.2323, -0.4378,  ..., -0.1354, -0.0446, -0.0160],
         [ 0.0512,  0.0360, -0.1019,  ..., -0.0164, -0.0712,  0.1906],
         [ 0.0230,  0.0742, -0.0304,  ..., -0.0839, -0.0225, -0.0232]]],
       device='cuda:0')), ('pooler_output', tensor([[-4.8702e-02,  7.8212e-02, -8.9386e-04,  1.4708e-02,  5.5133e-02,
          9.3036e-02,  7.1719e-02,  9.3593e-02, -7.8331e-02, -1.3360e-02,
         -1.8497e-02, -2.8504e-02, -2.5937e-02, -9.0535e-02, -3.8111e-02,
         -6.6021e-02, -9.6261e-03,  7.9695e-02,  1.4641e-02, -2.1324e-03,
         -4.4374e-02, -1.0318e-04,  1.0365e-01,  2.0434e-02, -5.1304e-02,
          3.9862e-02,  2.9633e-02,  1.0161e-01, -1.9505e-01,  3.7376e-02,
          2.6772e-02,  6.0044e-02, 

In [ ]:
model_output.last_hidden_state.shape

torch.Size([1, 42, 768])

In [ ]:
ind_of_text = 0
word_representation = model_output.last_hidden_state[ind_of_text][subtoken_start:subtoken_end].mean(dim=0)
word_representation.shape

torch.Size([768])

In [ ]:
import torch
from torch.utils.data.dataset import Dataset

class SSCDataset(Dataset):

    def __init__(self, data, tokenizer, label=1, label_field="y", device = "cuda"):
        self.data = data
        self.tokenizer = tokenizer
        self.label = label
        self.label_field = label_field
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        tokenization_con1 = self.tokenizer(item["context_1"], return_offsets_mapping=True)
        tokenization_con2 = self.tokenizer(item["context_2"], return_offsets_mapping=True)
        subtoken_start_1, subtoken_end_1 = indicate_subtokens(tokenization_con1, item['target_word_location_1']['char_start'], item['target_word_location_1']['char_end'])
        subtoken_start_2, subtoken_end_2 = indicate_subtokens(tokenization_con2, item['target_word_location_2']['char_start'], item['target_word_location_2']['char_end'])
        answer = dict()
        answer["input_ids_1"] = torch.tensor(tokenization_con1["input_ids"], dtype=torch.int64).to(self.device)
        answer["input_ids_2"] = torch.tensor(tokenization_con2["input_ids"], dtype=torch.int64).to(self.device)
        answer["attention_mask_1"] = tokenization_con1["attention_mask"]
        answer["attention_mask_2"] = tokenization_con2["attention_mask"]
        answer["subtoken_start_1"], answer["subtoken_end_1"] = torch.tensor(subtoken_start_1, dtype=torch.int64).to(self.device), torch.tensor(subtoken_end_1, dtype=torch.int64).to(self.device)
        answer["subtoken_start_2"], answer["subtoken_end_2"] = torch.tensor(subtoken_start_2, dtype=torch.int64).to(self.device), torch.tensor(subtoken_end_2, dtype=torch.int64).to(self.device)
        if "label" in item:
            answer[self.label_field] = torch.tensor(int(item["label"] == self.label), dtype=torch.int64).to(self.device)
        return answer

In [ ]:
train_data = dataset["train"]
train_dataset = SSCDataset(train_data, tokenizer)
for key, value in train_dataset[0].items():
    print(key, value, sep="\n")

input_ids_1
tensor([    5,   554,    17,    11, 10087,    16, 24714,  3168,    90,   121,
         2054,   233,    38,  3367,    58,  2063,    21,     3,   142,    16,
         1428,    21,     3,    38,  2264,  4097,    53,    33,  1975, 17467,
           77, 29771,     8,    21,     3,    38,   138,   133,     3,  4097,
          120,     6], device='cuda:0')
input_ids_2
tensor([    5,   160,    12,   856,    16, 11054, 11166, 10970,  7558,    52,
         4291,  3168,    90,     7,  2240,    29,    16, 17341,   345,  1151,
          244,     7,    76,   521,    21,     3,  6637,    19, 13939,    10,
           25, 16187,     9,     6], device='cuda:0')
attention_mask_1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask_2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
subtoken_start_1
tensor(6, device='cuda:0')
subtoken_end_1
tensor(9, dev

In [ ]:
dev_data = dataset["validation"]
dev_dataset = SSCDataset(dev_data, tokenizer)
for key, value in dev_dataset[0].items():
    print(key, value, sep="\n")

input_ids_1
tensor([    5, 23049,   108,    29,    17,    12,  5148,     7,    29,    13,
        10081,     7,    29,    16,  8038,     9,     6], device='cuda:0')
input_ids_2
tensor([    5, 23049,   108,    22,   701, 27455,    43,   278,    12,   177,
           40,    29,    19,  1491,   435,    10,     8,  1122,    12,    59,
            7,    49,    12,   443,    40,     8,   292,    15,    77, 17532,
            7,    15,    77,  2558,     9,     6], device='cuda:0')
attention_mask_1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask_2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
subtoken_start_1
tensor(0, device='cuda:0')
subtoken_end_1
tensor(3, device='cuda:0')
subtoken_start_2
tensor(0, device='cuda:0')
subtoken_end_2
tensor(3, device='cuda:0')
y
tensor(1, device='cuda:0')


## Dataloader

In [ ]:
import numpy as np
from torch.utils.data import DataLoader

def collate_fn(samples, dtype=torch.int64, device = "cuda"):
    device = samples[0]["input_ids_1"].device
    #lengths = [elem["input_ids"].shape[0] for elem in samples]
    L1 = max(elem["input_ids_1"].shape[0] for elem in samples)
    L2 = max(elem["input_ids_2"].shape[0] for elem in samples)
    # print(lengths, L)
    # to_pad = [torch.zeros(size=(L-length,), dtype=dtype).to(device) for length in lengths]
    padded_input_ids_1 = torch.stack([
        torch.cat([elem["input_ids_1"],
                   torch.zeros(size=(L1-len(elem["input_ids_1"]),), dtype=dtype).to(device)])
        for elem in samples
    ])
    padded_input_ids_2 = torch.stack([
        torch.cat([elem["input_ids_2"],
                   torch.zeros(size=(L2-len(elem["input_ids_2"]),), dtype=dtype).to(device)])
        for elem in samples
    ])
    # print([elem.shape for elem in padded_input_ids])
    return {"input_ids_1": padded_input_ids_1,
            "input_ids_2": padded_input_ids_2,
            "subtoken_start_1": torch.stack([elem["subtoken_start_1"] for elem in samples]),
            "subtoken_end_1": torch.stack([elem["subtoken_end_1"] for elem in samples]),
            "subtoken_start_2": torch.stack([elem["subtoken_start_2"] for elem in samples]),
            "subtoken_end_2": torch.stack([elem["subtoken_end_2"] for elem in samples]),
            "label": torch.stack([elem["y"] for elem in samples])}

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
for i, batch in enumerate(train_dataloader):
    if i >= 10:
        break
    print(batch["input_ids_1"].shape, batch["input_ids_2"].shape, batch["subtoken_start_1"].shape, batch["subtoken_end_1"].shape, batch["subtoken_start_2"].shape, batch["subtoken_end_2"].shape, batch["label"].shape)

torch.Size([8, 57]) torch.Size([8, 40]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])
torch.Size([8, 36]) torch.Size([8, 45]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])
torch.Size([8, 42]) torch.Size([8, 26]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])
torch.Size([8, 43]) torch.Size([8, 37]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])
torch.Size([8, 49]) torch.Size([8, 41]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])
torch.Size([8, 46]) torch.Size([8, 35]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])
torch.Size([8, 47]) torch.Size([8, 41]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])
torch.Size([8, 51]) torch.Size([8, 33]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8]) torch.Size([8])
torch.Size([8, 40]) torch.Size([8, 39]) 

##Создание модели

In [ ]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained("camembert-base").to("cuda")

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

## Взятие среднего значения из интервала

Код получает на вход трёхмерный тензор `input`, а также два списка `starts` и `ends` и возвращает двумерный тензор `output`, где $input[i,j] = \frac{1}{ends[i]-starts[i]}\sum\limits_{k=starts[i]}^{ends[i]-1]}input[i,k,j]$.

In [ ]:
import torch

data = torch.arange(24).reshape(2, 4, 3) # это как будто выдача Берта для батча
starts = torch.tensor([0, 1])
ends = torch.tensor([2, 4]) # то есть в первой строке усредняет вектора слов с номерами 0 и 1, во второй -- 1,2,3
column_indexes = torch.cumsum(torch.ones(size=data.shape[:2], dtype=int), dim=1)-1
selection_mask = torch.minimum(column_indexes >= starts[:,None], column_indexes < ends[:,None]) # true только на тех позициях, которые нужно брать
selected_data = torch.where(torch.unsqueeze(selection_mask, dim=-1), data, 0)
numerator = torch.sum(selected_data, dim=1)
denominator = ends-starts
result = numerator / denominator[:,None]
print(data)

print(result)

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17],
         [18, 19, 20],
         [21, 22, 23]]])
tensor([[ 1.5000,  2.5000,  3.5000],
        [18.0000, 19.0000, 20.0000]])


In [ ]:
import torch

def mean_output(embedding, starts, ends, device = "cuda"):
  # data = torch.arange(24).reshape(2, 4, 3) # это как будто выдача Берта для батча
  # starts = torch.tensor([0, 1])
  # ends = torch.tensor([2, 4]) # то есть в первой строке усредняет вектора слов с номерами 0 и 1, во второй -- 1,2,3
  column_indexes = (torch.cumsum(torch.ones(size=embedding.shape[:2], dtype=int), dim=1)-1).to(device)
  selection_mask = (torch.minimum(column_indexes >= starts[:,None], column_indexes < ends[:,None])).to(device) # true только на тех позициях, которые нужно брать
  selected_data = (torch.where(torch.unsqueeze(selection_mask, dim=-1), embedding, 0)).to(device)
  numerator = torch.sum(selected_data, dim=1)
  denominator = ends-starts
  result = numerator / denominator[:,None]
  return result

In [ ]:
device = "cuda"
for i, batch in enumerate(train_dataloader):
  if i >= 1:
    break
#batch = train_dataloader[0]
input_ids_1 = batch["input_ids_1"].to(device)
input_ids_2 = batch["input_ids_2"].to(device)
subtoken_start_1 = batch["subtoken_start_1"].to(device)
subtoken_end_1 = batch["subtoken_end_1"].to(device)
subtoken_start_2 = batch["subtoken_start_2"].to(device)
subtoken_end_2 = batch["subtoken_end_2"].to(device)
first_bert_output = bert_model(input_ids_1)["last_hidden_state"].to(device)
print(f"first_bert_output: {first_bert_output.shape}")
second_bert_output = bert_model(input_ids_2)["last_hidden_state"].to(device)
print(f"second_bert_output: {second_bert_output.shape}")
first_embedding = first_bert_output[:, subtoken_start_1].to(device)
print(f"first_embedding: {first_embedding.shape}")
second_embedding = second_bert_output[:, subtoken_start_2].to(device)
print(f"second_embedding: {second_embedding.shape}")
first_emb_mean = mean_output(first_embedding, subtoken_start_1, subtoken_end_1)
print(f"first_emb_mean: {first_emb_mean.shape}")
second_emb_mean = mean_output(second_embedding, subtoken_start_2, subtoken_end_2)
print(f"second_emb_mean: {second_emb_mean.shape}")
# full_embedding = torch.cat([first_embedding, second_embedding, first_embedding - second_embedding, first_embedding*second_embedding])
# print(f"full_embedding: {full_embedding.shape}")
full_embedding = torch.cat([first_emb_mean, second_emb_mean, first_emb_mean - second_emb_mean, first_emb_mean*second_emb_mean], dim = 1)
print(f"full_embedding: {full_embedding.shape}")


first_bert_output: torch.Size([8, 42, 768])
second_bert_output: torch.Size([8, 56, 768])
first_embedding: torch.Size([8, 8, 768])
second_embedding: torch.Size([8, 8, 768])
first_emb_mean: torch.Size([8, 768])
second_emb_mean: torch.Size([8, 768])
full_embedding: torch.Size([8, 3072])


In [ ]:
device = "cuda"
for i, batch in enumerate(train_dataloader):
  if i >= 1:
    break
#batch = train_dataloader[0]
input_ids_1 = batch["input_ids_1"].to(device)
input_ids_2 = batch["input_ids_2"].to(device)
subtoken_start_1 = batch["subtoken_start_1"].to(device)
subtoken_end_1 = batch["subtoken_end_1"].to(device)
subtoken_start_2 = batch["subtoken_start_2"].to(device)
subtoken_end_2 = batch["subtoken_end_2"].to(device)
first_bert_output = model(input_ids_1)["last_hidden_state"]
print(f"first_bert_output: {first_bert_output.shape}")
second_bert_output = model(input_ids_2)["last_hidden_state"]
print(f"second_bert_output: {second_bert_output.shape}")
first_embedding = first_bert_output[:, subtoken_start_1]
print(f"first_embedding: {first_embedding.shape}")
second_embedding = second_bert_output[:, subtoken_start_2]
print(f"second_embedding: {second_embedding.shape}")
full_embedding = torch.cat([first_embedding, second_embedding, first_embedding - second_embedding, first_embedding*second_embedding])
print(f"full_embedding: {full_embedding.shape}")

first_bert_output: torch.Size([8, 41, 768])
second_bert_output: torch.Size([8, 37, 768])
first_embedding: torch.Size([8, 8, 768])
second_embedding: torch.Size([8, 8, 768])
full_embedding: torch.Size([32, 8, 768])


In [ ]:
batch["label"]

tensor([1, 0, 0, 1, 0, 1, 0, 0], device='cuda:0')

In [ ]:
# import torch
# def forward(self, input_ids_1, input_ids_2, subtoken_start_1, subtoken_end_1, subtoken_start_2, subtoken_end_2, **kwargs):
#       input_ids_1 = input_ids_1.to(self.device)
#       input_ids_2 = input_ids_2.to(self.device)
#       subtoken_start_1 = subtoken_start_1.to(self.device)
#       subtoken_end_1 = subtoken_end_1.to(self.device)
#       subtoken_start_2 = subtoken_start_2.to(self.device)
#       subtoken_end_2 = subtoken_end_2.to(self.device)
#       first_bert_output = self.model(input_ids_1)["last_hidden_state"]
#       second_bert_output = self.model(input_ids_2)["last_hidden_state"]
#       first_embedding = first_bert_output[:, subtoken_start_1]
#       second_embedding = second_bert_output[:, subtoken_start_2]
#       full_embedding = torch.cat(first_embedding, second_embedding, first_embedding - second_embedding, first_embedding*second_embedding)

In [ ]:
import torch.nn as nn
from transformers.optimization import AdamW

class BasicTransformersClassificationModel(nn.Module):

    def __init__(self, model, labels_number, lr=1e-5, device="cpu", **kwargs):
        super(BasicTransformersClassificationModel, self).__init__()
        self.model = model
        self.labels_number = labels_number
        self.build_network(labels_number)
        # определяем функцию потерь
        self.log_softmax = nn.LogSoftmax(dim=-1)
        self.criterion = nn.NLLLoss(reduction="mean")
        self.sigmoid = nn.Sigmoid()
        # else:
        #     self.log_softmax = nn.Sigmoid() # sigma(x) = 1/ (1+exp(-x))
        #     self.criterion = nn.BCELoss(reduction="mean")
        self.device = device
        if self.device is not None:
            self.to(self.device)
        self.optimizer = AdamW(self.parameters(), lr=lr, weight_decay=0.01)

    @property
    def hidden_size(self):
        return self.model.config.hidden_size

    def forward(self, input_ids, **kwargs):
        raise NotImplementedError("You should implement forward pass in your derived class.")

    def train_on_batch(self, x, y):
        self.train()
        self.optimizer.zero_grad()
        loss = self._validate(x, y)
        loss["loss"].backward()
        self.optimizer.step()
        return loss

    def validate_on_batch(self, x, y):
        self.eval()
        with torch.no_grad():
            return self._validate(x, y)

    def _validate(self, x, y):
        if self.device is not None:
            y = y.to(self.device)
        #print(f"self(**x): {self(**x)}")
        batch_output = self(**x) #   self.forward(x) = self.__call__(x)
        # batch_output теперь словарь
        #print(batch_output["log_probs"])
        loss = self.criterion(batch_output["log_probs"], y)
        #print(batch_output["log_probs"])
        #print(y)
        batch_output["loss"] = loss
        return batch_output


class TransformersClassificationModel(BasicTransformersClassificationModel):

    def build_network(self, labels_number):
        self.proj_layer = torch.nn.Linear(self.hidden_size*4, self.labels_number)
        return self

    # def forward(self, input_ids, token_type_ids=None, **kwargs):
    #     input_ids = input_ids.to(self.device)
    #     if token_type_ids is not None:
    #         token_type_ids = token_type_ids.to(self.device)
    #     cls_output = self.model(input_ids, token_type_ids)["pooler_output"]
    #     logits = self.proj_layer(cls_output)
    #     log_probs = self.log_softmax(logits)
    #     if self.labels_number == 1:
    #         log_probs = log_probs[...,0]
    #         labels = (log_probs >= 0.5).int()
    #     else:
    #         _, labels = torch.max(log_probs, dim=-1)
    #     return {"log_probs": log_probs, "labels": labels}
    def forward(self, input_ids_1, input_ids_2, subtoken_start_1, subtoken_end_1, subtoken_start_2, subtoken_end_2, **kwargs):
        input_ids_1 = input_ids_1.to(self.device)
        input_ids_2 = input_ids_2.to(self.device)
        subtoken_start_1 = subtoken_start_1.to(self.device)
        subtoken_end_1 = subtoken_end_1.to(self.device)
        subtoken_start_2 = subtoken_start_2.to(self.device)
        subtoken_end_2 = subtoken_end_2.to(self.device)
        first_bert_output = self.model(input_ids_1)["last_hidden_state"].to(self.device)
        #print(f"first_bert_output: {first_bert_output.shape}")
        second_bert_output = self.model(input_ids_2)["last_hidden_state"].to(self.device)
        #print(f"second_bert_output: {second_bert_output.shape}")
        first_embedding = first_bert_output[:, subtoken_start_1].to(self.device)
        #print(f"first_embedding: {first_embedding.shape}")
        second_embedding = second_bert_output[:, subtoken_start_2].to(self.device)
        #print(f"second_embedding: {second_embedding.shape}")
        first_emb_mean = mean_output(first_embedding, subtoken_start_1, subtoken_end_1)
        #print(f"first_emb_mean: {first_emb_mean.shape}")
        second_emb_mean = mean_output(second_embedding, subtoken_start_2, subtoken_end_2)
        #print(f"second_emb_mean: {second_emb_mean.shape}")
        # full_embedding = torch.cat([first_embedding, second_embedding, first_embedding - second_embedding, first_embedding*second_embedding])
        # print(f"full_embedding: {full_embedding.shape}")
        full_embedding = torch.cat([first_emb_mean, second_emb_mean, first_emb_mean - second_emb_mean, first_emb_mean*second_emb_mean], dim = 1)
        #print(f"full_embedding: {full_embedding.shape}")
        logits = self.proj_layer(full_embedding)
        log_probs = self.log_softmax(logits)
        # log_probs = self.sigmoid(logits)
        if self.labels_number == 1:
            log_probs = log_probs[...,0]
            labels = (log_probs >= 0.5).int()
        else:
            _, labels = torch.max(log_probs, dim=-1)
        return {"log_probs": log_probs, "labels": labels}

In [ ]:
model = TransformersClassificationModel(
    bert_model, labels_number=2, device="cuda"
)
device = "cuda"
model.to(device)
# for name, elem in model.named_parameters():
#     print(name, elem.device, elem.shape)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


TransformersClassificationModel(
  (model): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
#batch = next(train_dataloader)
for i, batch in enumerate(train_dataloader):
  if i >= 1:
    break
labels = batch["label"]
print(labels)
for i in range(100):
    loss = model.train_on_batch(batch, labels)["loss"].item()
    if i < 5 or (i+1) % 10 == 0:
        print(i, loss)
    # break
print(model.validate_on_batch(batch, labels)["loss"].item())

tensor([1, 1, 0, 1, 1, 1, 0, 1], device='cuda:0')
0 0.6922681331634521
1 0.6860021352767944
2 0.68397456407547
3 0.6784807443618774
4 0.6758114695549011
9 0.6647688746452332
19 0.6311187744140625
29 0.6140806078910828
39 0.5878043174743652
49 0.5681137442588806
59 0.5525432229042053
69 0.5453988313674927
79 0.528301477432251
89 0.5192371606826782
99 0.5132120847702026
0.5032311677932739


#Обучение модели

In [ ]:
def update_metrics(metrics, batch_output, batch_labels):
    # код только для бинарных меток!!!!
    n_batches = metrics["n_batches"]
    metrics["loss"] = (metrics["loss"] * n_batches + batch_output["loss"].item()) / (n_batches + 1)
    metrics["n_batches"] += 1
    are_equal = (batch_output["labels"] == batch_labels).float()
    are_TP = are_equal * batch_labels
    are_FP = (1.0 - are_equal) * (1-batch_labels)
    are_FN = (1.0 - are_equal) * batch_labels
    metrics["correct"] += are_equal.sum().item()
    metrics["total"] += are_equal.shape[0]
    metrics["TP"] += are_TP.sum().item()
    metrics["FP"] += are_FP.sum().item()
    metrics["FN"] += are_FN.sum().item()
    metrics["accuracy"] = metrics["correct"] / max(metrics["total"], 1)
    metrics["P"] = metrics["TP"] / max(metrics["TP"]+metrics["FP"], 1)
    metrics["R"] = metrics["TP"] / max(metrics["TP"]+metrics["FN"], 1)
    metrics["F1"] = metrics["TP"] / max(metrics["TP"]+0.5*metrics["FN"]+0.5*metrics["FP"], 1)

In [ ]:
import tqdm

def do_epoch(model, dataloader, mode="validate", epoch=1):
    metrics = {"correct": 0, "total": 0, "loss": 0.0, "n_batches": 0,
               "TP": 0, "FP": 0, "FN": 0}
    func = model.train_on_batch if mode == "train" else model.validate_on_batch
    progress_bar = tqdm.notebook.tqdm(dataloader, leave=True)
    progress_bar.set_description(f"{mode}, epoch={epoch}")
    for batch in progress_bar:
        batch_answers = batch["label"]
        batch_output = func(batch, batch_answers)
        update_metrics(metrics, batch_output, batch_answers)
        postfix = {"loss": round(metrics["loss"], 4), "acc": round(100 * metrics["accuracy"], 2)}
        for key in ["P", "R", "F1"]:
            postfix[key] =  round(100 * metrics[key], 2)
        progress_bar.set_postfix(postfix)
    return metrics

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

bert_model = AutoModel.from_pretrained("camembert-base").to("cuda")
model = TransformersClassificationModel(bert_model, labels_number=2, lr = 1e-2, device="cuda")
model.to(device)
best_val_acc = 0.0
checkpoint = "checkpoint_best.pt"
for epoch in range(5):
    do_epoch(model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(model.state_dict(), checkpoint)
        # print("Saving ")
model.load_state_dict(torch.load(checkpoint))
do_epoch(model, dev_dataloader, mode="validate", epoch="evaluate")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/4929 [00:00<?, ?it/s]

  0%|          | 0/1074 [00:00<?, ?it/s]

  0%|          | 0/4929 [00:00<?, ?it/s]

  0%|          | 0/1074 [00:00<?, ?it/s]

...

  0%|          | 0/1074 [00:00<?, ?it/s]

  0%|          | 0/4929 [00:00<?, ?it/s]

  0%|          | 0/1074 [00:00<?, ?it/s]

  0%|          | 0/1074 [00:00<?, ?it/s]

{'correct': 4670.0,
 'total': 8588,
 'loss': 0.6910557660865615,
 'n_batches': 1074,
 'TP': 1406.0,
 'FP': 1030.0,
 'FN': 2888.0,
 'accuracy': 0.5437820214252446,
 'P': 0.577175697865353,
 'R': 0.3274336283185841,
 'F1': 0.41783060921248144}

In [ ]:
#lr = 1e-3 F-1: 0.407 R = 0.31 P = 0.577 acc = 0.542
#lr = 1e-5 F-1: 0.43 R = 0.35 P = 0.569 acc = 0.543
#lr = 1e-6 F-1: 0.52 R = 0.48 P = 0.576 acc = 0.564
#lr = 1e-7 F-1:0.17 R = 0.10
#lr = 1e-2 F1: 0.417 R: 0.327 P: 0.577 acc: 0.543


In [ ]:
test_data = dataset["test"]
test_dataset = SSCDataset(test_data, tokenizer)

In [ ]:
X = test_dataset
def predict_with_model(model, X, classes):
    model.eval()
    dataloader = DataLoader(X, device=model.device)
    answer = [None] * len(X)
    for batch in dataloader:
        with torch.no_grad():
            batch_answer = model(**batch)
        # только для бинарной классификации
        # labels = (batch_answer["probs"] >= 0.5).int().cpu().numpy()
        labels = (batch_answer["labels"]).cpu().numpy()
        for i, prob, label in zip(batch["indexes"], batch_answer["log_probs"], labels):
            # print("PROB", prob)
            answer[i] = {'label': classes[label], 'prob': prob.item()}
    return answer

In [ ]:
predicted = predict_with_model(model, dev_dataset, classes=["not_entailment", "entailment"])
for i in range(10,20):
    print(dev_data[i]["premise"])
    print(dev_data[i]["hypothesis"])
    print(dev_data[i]["label"], predicted[i]['label'], f"{100*predicted[i]['prob']:.2f}")
    print("")

# Замена на Roberta

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("abhilash1910/french-roberta")
tokens = tokenizer.tokenize(dataset["train"][0]["context_1"])
tokens

tokenizer_config.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/512k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/298k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

['Comme',
 'Ġl',
 "'",
 'indique',
 'Ġle',
 'ĠSh',
 'u',
 'ow',
 'en',
 'ĠJ',
 'iez',
 'i',
 'Ġ(',
 'dans',
 'Ġson',
 'Ġcommentaire',
 'Ġ',
 'ä',
 '»',
 'İ',
 'æ',
 'Ī',
 'Ĭ',
 'ä',
 '¸',
 'ģ',
 'è',
 'ģ',
 '²',
 '),',
 'Ġle',
 'ĠcaractÃ¨re',
 'Ġ',
 'æ',
 'Ī',
 'Ĳ',
 'Ġ(',
 'ch',
 'Ã©n',
 'g',
 ')',
 'Ġa',
 'Ġcertainement',
 'ĠhÃ©ritÃ©',
 'Ġsa',
 'Ġpron',
 'on',
 'ciation',
 'Ġde',
 'Ġ',
 'ä',
 '¸',
 'ģ',
 'Ġ(',
 'z',
 'h',
 'Ä',
 'ĵ',
 'ng',
 ').']

In [ ]:
len(tokens)

60

In [ ]:
item = dataset["train"][0]
tokenization = tokenizer(item["context_1"], return_offsets_mapping=True)
for key, value in tokenization.items():
    print(key, value)

input_ids [0, 2029, 264, 11, 15390, 292, 2712, 89, 2419, 265, 541, 4427, 77, 443, 7096, 435, 9519, 225, 165, 124, 241, 167, 235, 237, 165, 121, 228, 169, 228, 115, 960, 292, 4795, 225, 167, 235, 243, 443, 337, 941, 75, 13, 270, 4514, 19576, 445, 3811, 266, 10615, 268, 225, 165, 121, 228, 443, 94, 76, 133, 246, 17804, 847, 2]
attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
offset_mapping [(0, 0), (0, 5), (6, 7), (7, 8), (8, 15), (16, 18), (19, 21), (21, 22), (22, 24), (24, 26), (27, 28), (28, 31), (31, 32), (33, 34), (34, 38), (39, 42), (43, 54), (55, 55), (55, 56), (55, 56), (55, 56), (56, 57), (56, 57), (56, 57), (57, 58), (57, 58), (57, 58), (58, 59), (58, 59), (58, 59), (59, 61), (62, 64), (65, 74), (75, 75), (75, 76), (75, 76), (75, 76), (77, 78), (78, 80), (80, 82), (82, 83), (83, 84), (85, 86), (87, 99), (100, 106), (107, 109), 

In [ ]:
print(len(tokenization["input_ids"]))

62


In [ ]:
train_data = dataset["train"]
train_dataset_roberta = SSCDataset(train_data, tokenizer)
for key, value in train_dataset_roberta[0].items():
    print(key, value, sep="\n")

input_ids_1
tensor([    0,  2029,   264,    11, 15390,   292,  2712,    89,  2419,   265,
          541,  4427,    77,   443,  7096,   435,  9519,   225,   165,   124,
          241,   167,   235,   237,   165,   121,   228,   169,   228,   115,
          960,   292,  4795,   225,   167,   235,   243,   443,   337,   941,
           75,    13,   270,  4514, 19576,   445,  3811,   266, 10615,   268,
          225,   165,   121,   228,   443,    94,    76,   133,   246, 17804,
          847,     2], device='cuda:0')
input_ids_2
tensor([    0,    40,   280,  2315,   292,   416,   484,  4183,   498, 22579,
         3132,  7229,    89,  2419,   265,    16,  4933,   359,   292,   465,
         6795,  4183, 21730,   346,    16,   341,   383,   225,   168,   112,
          227, 20240,   306,   299,  1017,  1208,    87,   326, 29114,   272,
           18,     2], device='cuda:0')
attention_mask_1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,